In [1]:
import warnings
warnings.filterwarnings('ignore')

import geopandas as gpd
import pandas as pd
import shapely
from shapely.geometry import Polygon, Point
import geopy.distance
import pandana
import numpy as np
import requests
import folium

In [2]:
stroke_facs = pd.read_csv('stroke_facs_latest.csv')

In [3]:
stroke_facs = stroke_facs[['Name_English','Facility name-VN','Type_name','address','longitude','latitude',
                           'pro_name_e','dist_name_e']].reset_index()


stroke_facs.rename(columns={'index':'fac_id'},inplace=True)

In [4]:
stroke_facs['longitude'] = stroke_facs['longitude'].astype(float)
stroke_facs['latitude'] = stroke_facs['latitude'].astype(float)


In [5]:
stroke_facs.head(2)

,fac_id,Name_English,Facility name-VN,Type_name,address,longitude,latitude,pro_name_e,dist_name_e
0,0,Saint Paul Municipal General Hospital,Bệnh viện đa khoa Xanh Pôn,provincial general hospital,"12 Chu Văn An, Phường Điện Biên, Quận Ba Đình,...",105.835586,21.031610,Ha Noi,Ba Dinh District
1,1,Viet Duc (University) Hospital (CS1,Bệnh viện Hữu nghị Việt Đức (CS1 Hà Nội),Central specialized hospital,"40 Tràng Thi, P. Hàng Bông, Q. Hoàn Kiếm, Hà Nội",105.845353,21.029563,Ha Noi,Hoan Kiem District


In [6]:
len(stroke_facs)

106

In [7]:
population = pd.read_csv(r'ppp_VNM_2020_1km_Aggregated_UNadj.csv').reset_index()
population.columns = ['ID','xcoord','ycoord','household_count']
population['xcoord'] = population['xcoord'].round(2)
population['ycoord'] = population['ycoord'].round(2)

population = population.groupby(['xcoord','ycoord'])['household_count'].sum().reset_index().reset_index()
population['household_count'] = population['household_count'].round()
population.columns = ['ID','xcoord','ycoord','population']

population_worldpop = gpd.GeoDataFrame(population, 
                                       geometry=gpd.points_from_xy(x=population.xcoord, 
                                                                   y=population.ycoord))

print('Total Population:',round(population_worldpop['population'].sum()/1000000,2),'million')

Total Population: 97.34 million


In [8]:
len(population_worldpop)

283557

In [ ]:
%%time

facebook_pop_csv = pd.read_csv('vnm_general_2020.csv')
population = facebook_pop_csv[['latitude','longitude','vnm_general_2020']].reset_index()
population.columns = ['ID','ycoord','xcoord','population']

population_meta = gpd.GeoDataFrame(population, 
                                       geometry=gpd.points_from_xy(x=population.xcoord, 
                                                                   y=population.ycoord))
gadm_vnm = gpd.read_file('gadm41_VNM_shp/gadm41_VNM_0.shp')
population_meta = gpd.sjoin(population_meta, gadm_vnm, op='within')
print('Total Population:',round(population_meta['population'].sum()/1000000,2),'million')


In [ ]:
len(population_meta)

In [9]:
access_token = "Enter MapBox API Key Here"

In [10]:
def get_isochrone(df,minutes_list,access_token,mode):
    longitude = df['longitude']
    latitude = df['latitude']
    query = """https://api.mapbox.com/isochrone/v1/mapbox/"""
    query = query+mode+'/'
    query = query+str(longitude)+','+str(latitude)+'?'
    query = query+'contours_minutes='+minutes_list
    query = query+'&polygons=true&access_token='
    query = query+access_token
    req_return = (requests.get(query).json())
    
    if('code' in req_return):
        if (req_return['code']=='NoSegment'):
            print('No Segment')
        else:
            print(req_return)     
    else:
        #print(req_return)     
        return(req_return['features'])

In [11]:
def get_pop_count(population_df,x):
    pop_count = population_df[population_df['ID'].isin(x)]['population'].sum()
    return pop_count

In [ ]:
%%time

stroke_facs['isochrone_30mins'] = stroke_facs[['longitude','latitude']].apply(get_isochrone,
                                                                                   minutes_list="30",
                                                                                   access_token=access_token,
                                                                                   mode='driving',
                                                                                   axis=1)

stroke_facs['isochrone_60mins'] = stroke_facs[['longitude','latitude']].apply(get_isochrone,
                                                                                   minutes_list="60",
                                                                                   access_token=access_token,
                                                                                   mode='driving',
                                                                                   axis=1)

stroke_facs['30mins'] = stroke_facs['isochrone_30mins'].apply(lambda x: x[0]['geometry'])
stroke_facs['30mins'] = stroke_facs['30mins'].apply(lambda x:Polygon(x['coordinates'][0]))

stroke_facs['60mins'] = stroke_facs['isochrone_60mins'].apply(lambda x: x[0]['geometry'])
stroke_facs['60mins'] = stroke_facs['60mins'].apply(lambda x:Polygon(x['coordinates'][0]))


In [13]:
def get_population_within_vector(vector_polygon,vector_layer):
    pip_mask = vector_layer.within(vector_polygon)
    pip_data = vector_layer.loc[pip_mask]
    return(list(pip_data['ID'].unique()))


In [12]:
## Change Population Here

population_selected = population_worldpop

In [ ]:
%%time

df_30mins = stroke_facs[['30mins','fac_id']]
df_30mins.columns = ['geometry','fac_id']
df_30mins = gpd.GeoDataFrame(df_30mins)
join30mins = gpd.sjoin(population_selected[['ID','geometry']],df_30mins,predicate='within')[['ID','fac_id']]
join30mins = join30mins.groupby('fac_id')['ID'].agg(list).reset_index()
join30mins['ID'] = join30mins['ID'].apply(lambda x:list(set(x)))
join30mins.columns = ['fac_id','ID_30mins']


In [ ]:
%%time

df_60mins = stroke_facs[['60mins','fac_id']]
df_60mins.columns = ['geometry','fac_id']
df_60mins = gpd.GeoDataFrame(df_60mins)
join60mins = gpd.sjoin(population_selected[['ID','geometry']],df_60mins,predicate='within')[['ID','fac_id']]
join60mins = join60mins.groupby('fac_id')['ID'].agg(list).reset_index()
join60mins['ID'] = join60mins['ID'].apply(lambda x:list(set(x)))
join60mins.columns = ['fac_id','ID_60mins']

In [ ]:
stroke_facs = pd.merge(stroke_facs,join30mins, on='fac_id')
stroke_facs = pd.merge(stroke_facs,join60mins, on='fac_id')

In [ ]:
stroke_facs.head(2)

In [ ]:
start_coords = (14.0583, 108.2772)
folium_map = folium.Map(location=start_coords, zoom_start=5)

test_ids = stroke_facs

for i in range(0,len(test_ids)):
    folium.Marker([test_ids.iloc[i]['latitude'], test_ids.iloc[i]['longitude']],
                        color='blue',popup=test_ids.iloc[i]['Name_English']).add_to(folium_map)
    
    geo_j = folium.GeoJson(data=test_ids.iloc[i]['60mins'],style_function=lambda x:{'color': 'blue'})
    folium.Popup(test_ids.iloc[i]['Name_English']).add_to(geo_j)
    geo_j.add_to(folium_map)
    
    geo_j = folium.GeoJson(data=test_ids.iloc[i]['30mins'],style_function=lambda x:{'color': 'purple'})
    folium.Popup(test_ids.iloc[i]['Name_English']).add_to(geo_j)
    geo_j.add_to(folium_map)
    

# Define the URL template for the Mapbox basemap
mapbox_url = f'https://api.mapbox.com/styles/v1/mapbox/{{id}}/tiles/{{z}}/{{x}}/{{y}}?access_token={access_token}'
mapbox_basemap = folium.TileLayer(tiles=mapbox_url, attr='Mapbox', name='Mapbox Bright', max_zoom=20, id='mapbox/light-v10')

# Add the Mapbox basemap as a baselayer to the map
mapbox_basemap.add_to(folium_map)

# Create a custom HTML legend
legend_html = '''
     <div style="position: fixed; 
                 bottom: 50px; left: 50px; width: 180px; height: 90px; 
                 border:2px solid grey; z-index:9999; font-size:14px;
                 background-color: white;
                 ">&nbsp;<b>Legend (Isodistances)</b><br>
         &nbsp;<i class="fa fa-square" style="color:blue"></i>&nbsp; 60 mins driving<br>
         &nbsp;<i class="fa fa-square" style="color:purple"></i>&nbsp; 30 mins driving<br>
      </div>
     '''

# Add the custom legend as a separate layer to the map
folium_map.get_root().html.add_child(folium.Element(legend_html))

heading_html = '''
     <div style="position: absolute; 
                 top: 50px; left: 50px; width: 220px; height: 50px; 
                 border: none; z-index:9999; font-size: 15px;
                 background-color: rgba(255, 255, 255, 0.6);
                 ">&nbsp;<b>Stroke Facility Catchment Area </b><br>
      </div>
     '''

# Add the custom heading as a separate layer to the map
folium_map.get_root().html.add_child(folium.Element(heading_html))

folium_map


In [ ]:
%%time

list_pop_ids = list(stroke_facs['ID_60mins'].values)
list_pop_ids = [item for sublist in list_pop_ids for item in sublist]
pop_with_access = list(set(list_pop_ids))

(population_selected[population_selected['ID'].isin(pop_with_access)]['population'].sum()*100/population_selected['population'].sum()).round()


In [ ]:
%%time

list_pop_ids = list(stroke_facs['ID_30mins'].values)
list_pop_ids = [item for sublist in list_pop_ids for item in sublist]
pop_with_access = list(set(list_pop_ids))

(population_selected[population_selected['ID'].isin(pop_with_access)]['population'].sum()*100/population_selected['population'].sum()).round()


In [14]:
potential_locations = gpd.read_file('10kmgrid.geojson')
potential_locations = potential_locations.reset_index()[['index','geometry']]
potential_locations['latitude'] = potential_locations['geometry'].y
potential_locations['longitude'] = potential_locations['geometry'].x
potential_locations.columns = ['Hosp_ID','geometry','latitude','longitude']

In [15]:
potential_locations.head(2)

,Hosp_ID,geometry,latitude,longitude
0,0,POINT (102.19129 22.36409),22.364086,102.191293
1,1,POINT (102.26897 22.46075),22.460746,102.268968


In [16]:
%%time

potential_locations['isochrone_30mins'] = potential_locations[['longitude','latitude']].apply(get_isochrone,
                                                                                   minutes_list="30",
                                                                                   access_token=access_token,
                                                                                   mode='driving',
                                                                                   axis=1)

potential_locations['isochrone_60mins'] = potential_locations[['longitude','latitude']].apply(get_isochrone,
                                                                                   minutes_list="60",
                                                                                   access_token=access_token,
                                                                                   mode='driving',
                                                                                   axis=1)

potential_locations['30mins'] = potential_locations['isochrone_30mins'].apply(lambda x: x[0]['geometry'])
potential_locations['30mins'] = potential_locations['30mins'].apply(lambda x:Polygon(x['coordinates'][0]))

potential_locations['60mins'] = potential_locations['isochrone_60mins'].apply(lambda x: x[0]['geometry'])
potential_locations['60mins'] = potential_locations['60mins'].apply(lambda x:Polygon(x['coordinates'][0]))


CPU times: user 1min 56s, sys: 11.9 s, total: 2min 8s
Wall time: 32min 35s


In [17]:
%%time

df_30mins = potential_locations[['30mins','Hosp_ID']]
df_30mins.columns = ['geometry','Hosp_ID']
df_30mins = gpd.GeoDataFrame(df_30mins)
join30mins = gpd.sjoin(population_selected[['ID','geometry']],df_30mins,predicate='within')[['ID','Hosp_ID']]
join30mins = join30mins.groupby('Hosp_ID')['ID'].agg(list).reset_index()
join30mins['ID'] = join30mins['ID'].apply(lambda x:list(set(x)))
join30mins.columns = ['Hosp_ID','ID_30mins']


CPU times: user 1.02 s, sys: 114 ms, total: 1.13 s
Wall time: 1.17 s


In [18]:
%%time

df_60mins = potential_locations[['60mins','Hosp_ID']]
df_60mins.columns = ['geometry','Hosp_ID']
df_60mins = gpd.GeoDataFrame(df_60mins)
join60mins = gpd.sjoin(population_selected[['ID','geometry']],df_60mins,predicate='within')[['ID','Hosp_ID']]
join60mins = join60mins.groupby('Hosp_ID')['ID'].agg(list).reset_index()
join60mins['ID'] = join60mins['ID'].apply(lambda x:list(set(x)))
join60mins.columns = ['Hosp_ID','ID_60mins']

CPU times: user 3.05 s, sys: 462 ms, total: 3.51 s
Wall time: 3.55 s


In [19]:
potential_locations = pd.merge(potential_locations,join30mins, on='Hosp_ID')
potential_locations = pd.merge(potential_locations,join60mins, on='Hosp_ID')

In [20]:
%%time

list_pop_ids = list(potential_locations['ID_60mins'].values)
list_pop_ids = [item for sublist in list_pop_ids for item in sublist]
pop_with_access = list(set(list_pop_ids))

(population_selected[population_selected['ID'].isin(pop_with_access)]['population'].sum()*100/population_selected['population'].sum()).round()


CPU times: user 187 ms, sys: 38.8 ms, total: 226 ms
Wall time: 225 ms


95.0

In [21]:
%%time

list_pop_ids = list(potential_locations['ID_30mins'].values)
list_pop_ids = [item for sublist in list_pop_ids for item in sublist]
pop_with_access = list(set(list_pop_ids))

(population_selected[population_selected['ID'].isin(pop_with_access)]['population'].sum()*100/population_selected['population'].sum()).round()


CPU times: user 78.7 ms, sys: 6.64 ms, total: 85.4 ms
Wall time: 84 ms


86.0

In [22]:
potential_locations.head(2)

,Hosp_ID,geometry,latitude,longitude,isochrone_30mins,isochrone_60mins,30mins,60mins,ID_30mins,ID_60mins
0,0,POINT (102.19129 22.36409),22.364086,102.191293,"[{'properties': {'fill-opacity': 0.33, 'fillCo...","[{'properties': {'fill-opacity': 0.33, 'fillCo...","POLYGON ((102.173293 22.512189, 102.171293 22....","POLYGON ((102.275293 22.613618, 102.269829 22....","[5, 6, 9, 10, 11, 12, 20, 33, 163, 47, 48, 60,...","[2, 4, 5, 6, 517, 8, 9, 10, 11, 12, 518, 20, 2..."
1,1,POINT (102.26897 22.46075),22.460746,102.268968,"[{'properties': {'fill-opacity': 0.33, 'fillCo...","[{'properties': {'fill-opacity': 0.33, 'fillCo...","POLYGON ((102.280969 22.504611, 102.279238 22....","POLYGON ((102.292969 22.504662, 102.277145 22....","[320, 129, 258, 291, 196, 356, 199, 200, 201, ...","[517, 518, 11, 19, 20, 21, 32, 46, 47, 562, 56..."
